# Analysis Tutorial

This tutorial demonstrates how to analyze and interpret the outputs from Connectomemapper 3. In particular it will tell you how to:

- Get the list of connectome files with [Pybids](https://pypi.org/project/pybids/)
- Read the .tsv connectome files with [Networkx](https://pypi.org/project/networkx/) and [Pandas](https://pandas.pydata.org/)
- Read the .gpickle files with Networkx
- Read the .mat files with [Scipy](https://scipy.org/)
- Compute the connectome harmonics with [PyGSP](https://pygsp.readthedocs.io/en/stable/index.html)
- Visualize the harmonics with the plot functions of [Nilearn](https://nilearn.github.io/stable/index.html)

## Setup instructions

If you want to reproduce all the results of this notebook on your side, a conda ``environment.yml`` file can be downloaded at the following link: [tutorial_environment.yml](tutorial_environment.yml). The original ``.ipynb`` notebook file can be downloaded at the following link: [analysis_tutorial.ipynb](analysis_tutorial.ipynb).

Once you have downloaded the conda environment file, install the environment as follows:
```bash
$ conda create env -f /path/to/downloaded/analysis_tutorial.yml
```
This will install all the packages needed to run this notebook including jupyter lab.

You can then activate it, go to the directory where you downloaded the ``analysis_tutorial.ipynb``, and launch jupyter lab as follows:

```bash
$ cd /directory/of/downloaded/analysis_tutorial.ipynb/
$ conda activate cmp3-tutorial
$ jupyter lab
```

You are ready to open and interact with the notebook!

## Loading the external python packages

In [ ]:
# General
import os
import sys

# Dataset management
import datalad.api as dl

# Data handling
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.io as sio

# BIDS dataset handling
from bids import BIDSLayout

# Network / Graph
import pygsp
import networkx as nx

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import nilearn
from nilearn import plotting, image, datasets

## Loading the connectome files

For demonstration, we are going to use the VEPCON dataset, available on [Open Neuro](https://openneuro.org/datasets/ds003505/versions/1.0.2) that already contains output from Connectomemapper. A full description of the dataset can be found in [Pascucci, Tourbier, et al. 2022].

[Pascucci, Tourbier, et al. 2022]: https://doi.org/10.1038/s41597-021-01116-1

In [ ]:
# download example dataset with datalad
# uncomment next line once the new dataset is up
#vepcon_data = dl.install(path=os.path.join("..", "..", "example_data"),
#                         source="https://github.com/OpenNeuroDatasets/ds003505.git")

As the dataset is in [BIDS](https://bids.neuroimaging.io/), we can use [Pybids](https://bids-standard.github.io/pybids/) to help us with the task of interacting with the files of the dataset.

In [ ]:
#layout = BIDSLayout(vepcon_data.path)
# add derivative folder containing the connectivity matrices
#layout.add_derivatives(os.path.join(vepcon_data.path, "derivatives", "cmp-v3.0.0-beta-RC1"))

In [ ]:
#layout = BIDSLayout(vepcon_data.path)
path = os.path.join("/home", "localadmin", "data", "ds003505-sample")
layout = BIDSLayout(path)
# add derivative folder containing the connectivity matrices
#layout.add_derivatives(os.path.join(vepcon_data.path, "derivatives", "cmp-v3.0.0-beta-RC1"))
layout.add_derivatives(os.path.join(path, "derivatives", "cmp-v3.0.1"))

Now we can easily query for the filenames of the files we are interested in using `layout.get`. We will ask for the connectivity matrix of subject 01, scale 3, in tsv format. It will be returned as a list of file paths (in this case containing just one element).

In [ ]:
conn_tsv_scale3 = layout.get(subject='01', extension='tsv',
                             suffix='connectivity', return_type='filename',
                             res='scale3')
conn_tsv_scale3

We can then use [Pandas](https://pandas.pydata.org/) to read the file and display it as a table.

In [ ]:
#edges = pd.read_csv(con_files[0], delimiter="\t")
edges = pd.read_csv(conn_tsv_scale3[0], delimiter="\t")
edges.head()

Using [Networkx](https://networkx.org/documentation/stable/index.html), we can convert this table to a network graph. From that, we can convert individual measures to a [Numpy](https://numpy.org/) array. The array format is especially useful, as it allows us to plot the edge weights easily.

In [ ]:
G = nx.from_pandas_edgelist(edges, edge_attr=True)
A_fiber_density = nx.to_numpy_array(G, weight="fiber_density")

Alternatively, we can also load the matrices in network format, by reading the gpickle files using Networkx:

In [ ]:
conn_gpickle_scale3 = layout.get(subject='01', extension='gpickle',
                                 suffix='connectivity',
                                 return_type='filename',
                                 res='scale3')
G = nx.read_gpickle(conn_gpickle_scale3[0])  # same format as with tsv

...or load the .mat files with [Scipy](https://scipy.org/):

In [ ]:
conn_mat_scale3 = layout.get(subject='01', extension='mat',
                             suffix='connectivity', return_type='filename',
                             res='scale3')
A_mat = sio.loadmat(conn_mat_scale3[0])

The adjacency matrices here can be accessed as followss:

In [ ]:
A_mat["sc"]["fiber_density"][0][0]

## Plotting the adjacency matrices

Let's plot some of those adjacency matrices using [Matplotlib](https://matplotlib.org/) and [Seaborn](http://seaborn.pydata.org/index.html):

In [ ]:
cols_to_plot = ["number_of_fibers", "fiber_length_mean",
                "fiber_proportion", "fiber_density",
                "FA_mean", "normalized_fiber_density"]

fig, axs = plt.subplots(3,2, figsize=(12,15))
axs = axs.flatten()
for c, ax in zip(cols_to_plot, axs):
    A = nx.to_numpy_array(G, weight=c)
    sns.heatmap(A, ax=ax)
    ax.set_title(c)

## Graph signal processing with structural connectivity and visualization

The package [PyGSP](https://pygsp.readthedocs.io/en/stable/index.html) offers a range of graph signal processing tools we can use on our structural connectivity data. In particular, we can do an eigendecomposition of the [graph Laplacian](https://en.wikipedia.org/wiki/Laplacian_matrix) to get the Fourier basis - the connectome harmonics.

Even though it is possible to also do this for subcortical regions, for the sake of plotting it is easier just to work with the cortical regions. To identify those, we need the parcellation labels.

In [ ]:
# UPDATE PATH ONCE AVAILABLE
anat_folder = "/home/localadmin/data/ds003505-sample/derivatives/cmp-v3.0.1/sub-01/anat"
# read label tsv file
label_path = os.path.join(anat_folder, "sub-01_atlas-L2018_res-scale3_dseg.tsv")
labels = pd.read_csv(label_path, sep="\t", index_col=0)
# reset index to start at 0
labels.reset_index(inplace=True)
# select cortex labels
labels_ctx = labels["name"][[n.startswith("ctx") for n in labels["name"]]].copy()
idx = list(labels_ctx.index)
# select rows with cortical areas
#A_fd_ctx = A_fiber_density[idx]
A = nx.to_numpy_array(G, weight="FA_mean")
A_fd_ctx = A[idx]
# select columns with cortical areas
A_fd_ctx = A_fd_ctx[:,idx]

In [ ]:
A_fd_ctx.shape

Now we can compute the harmonics:

In [ ]:
np.fill_diagonal(A_fd_ctx, 0)  # PyGSP does not support self-loops
G_fd = pygsp.graphs.Graph(A_fd_ctx)  # PyGSP graph
G_fd.compute_laplacian(lap_type="normalized")
G_fd.compute_fourier_basis()  # compute connectome harmonics

The harmonics have the same dimensions as our original adjacency matrix.

In [ ]:
G_fd.U.shape

Each column contains one basis vector.

### Basic visualization with ``Nilearn``

[Nilearn](https://nilearn.github.io/stable/index.html) offers a quick and easy way to plot them using `plot_markers`. For this, we need the center coordinates of each region in the parcellation in MNI space. For your convenience, they have been already computed and can be easily retrieved with the `get_lausanne2018_parcellation_mni_coords(scale)` utility function of CMP3.

In [ ]:
from cmtklib.data.parcellation.util import get_lausanne2018_parcellation_mni_coords

In [ ]:
# load coordinates with the utility function provided by CMP3
coords_ctx = get_lausanne2018_parcellation_mni_coords('scale3')
# plot
plotting.plot_markers(G_fd.U[:,1], coords_ctx)

### Advanced visualization with ``Nilearn``

A prettier version is to plot the connectome harmonics on a brain surface using Nilearn `plot_surf_roi()`. For your convenience, a multiple view plot can be easily generated and saved with the `plot_lausanne2018_surface_ctx()` of the `cmtklib.data.parcellation.viz` module of CMP3, by specifying the scale to use.

These figures take a few minutes to generate, so you might need to be a bit patient.

In [ ]:
from cmtklib.data.parcellation.viz import plot_lausanne2018_surface_ctx

In [ ]:
plot_lausanne2018_surface_ctx(G_fd.U[:,1], scale='scale3', save_fig=True)

Pretty, right? This concludes the tutorial.

## Want to learn more about connectome harmonics?

Here are some references:

- Human brain networks function in connectome-specific harmonic waves (Atasoy et al., 2016, [link](https://www.nature.com/articles/ncomms10340)): Landmark paper that first applied graph signal processing to brain connectivity.

- Functional harmonics reveal multi-dimensional basis functions underlying cortical organization (Glomb et al., 2021, [link](https://doi.org/10.1016/j.celrep.2021.109554)): Connectome harmonics of functional connectivity.

- The connectome spectrum as a canonical basis for a sparse representation of fast brain activity (Rué-Queralt et al., 2021, [link](10.1016/j.neuroimage.2021.118611)): EEG and connectome harmonics.